In [13]:
import numpy as np
import pandas as pd
import tabula
import os

In [21]:
pd.set_option('display.max_rows',None)

# Annual fire #s and acres - wild and Rx - by State
Methods for cleaning determined in separate shitshow of a notebook, final functions here.

* All States of interest contained on pages 1, 2, 3, 7, 8, 10 of [NICC Redbooks](https://www.nifc.gov/fire-information/statistics) for years 2010-2013.
    * p1, p2-8, and p10 all require separate treatment because tabula likes to make you work for it
    * function for each page defined separately before combining, because possible that the pdfs per year are _slightly_ different enough to break the combined function; if have to resort to processing individual documents manually, easier to do with functions broken out

* Format slightly changes in 2014; new function required.

In [14]:
west_states = ['CA', 'OR', 'WA', 'AZ', 'NV', 'UT', 'ID']

### p1 methods

In [15]:
#define function for p1 cleaning.  
#all other pages way worse, but in the same way so can clean together

def p1_clean(filename):
    west_states = ['CA', 'OR', 'WA', 'AZ', 'NV', 'UT', 'ID']
    table = tabula.read_pdf(filename, pages=1)
    
    df = table[0].copy()
    df.rename(columns = {'Unnamed: 0': 'state', 'Unnamed: 1':'agency', 'Wildland':'fires_acres', 'Unnamed: 2': 'rx_fires', 'Rx': 'rx_acres'}, inplace=True)
    df.drop(index=0, inplace=True)
    df[['wildfires', 'wild_acres']] = df['fires_acres'].str.split(expand=True)
    df.drop('fires_acres', axis=1, inplace=True)
    df['state'] = df['state'].fillna(method='ffill')
    df = df.loc[df['state'].isin(west_states)]
    
    return df

### p2-8 methods

In [16]:
# define function for p2-9 cleaning

def p2to9_clean(filename):
    west_states = ['CA', 'OR', 'WA', 'AZ', 'NV', 'UT', 'ID']
    table = tabula.read_pdf(filename, pages=1)
    
    df = table[0].copy()
    
    df['Wildland'] = df['Wildland'].fillna('Totals')
    df.drop(columns=['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 10'], inplace=True)
    
    df['Rx'].fillna(df['Unnamed: 3'], inplace=True)
    df['Unnamed: 4'].fillna(df['Unnamed: 5'], inplace=True)
    df['Unnamed: 6'].fillna(df['Unnamed: 7'], inplace=True)
    df['Unnamed: 8'].fillna(df['Unnamed: 9'], inplace=True)
    
    df.drop(columns=['Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7', 'Unnamed: 9'], inplace=True)
    
    df.rename(columns={'Unnamed: 0':'state', 'Wildland': 'agency', 'Rx':'wildfires',
                     'Unnamed: 4':'wild_acres', 'Unnamed: 6':'rx_fires', 
                      'Unnamed: 8': 'rx_acres'}, inplace=True)
    
    df.drop(index=0, inplace=True)
    
    df['state'] = df['state'].fillna(method='ffill')
    
    df = df.loc[df['state'].isin(west_states)]
    
    return df

In [17]:
# define function for p10 cleaning: ok very few changes from p2 I overreacted

def p10_clean(filename):
    west_states = ['CA', 'OR', 'WA', 'AZ', 'NV', 'UT', 'ID']
    table = tabula.read_pdf(filename, pages=1)
    
    df = table[0].copy()
    
    df['Unnamed: 1'] = df['Unnamed: 1'].fillna(df['Rx'])
    df.drop(columns=['Rx', 'Wildland', 'Unnamed: 10'], inplace=True)
    
    df['Unnamed: 2'].fillna(df['Unnamed: 3'], inplace=True)
    df['Unnamed: 4'].fillna(df['Unnamed: 5'], inplace=True)
    df['Unnamed: 6'].fillna(df['Unnamed: 7'], inplace=True)
    df['Unnamed: 8'].fillna(df['Unnamed: 9'], inplace=True)
    
    df.drop(columns=['Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7', 'Unnamed: 9'], inplace=True)
    
    df.rename(columns={'Unnamed: 0':'state', 'Unnamed: 1': 'agency', 'Unnamed: 2':'wildfires',
                     'Unnamed: 4':'wild_acres', 'Unnamed: 6':'rx_fires', 
                      'Unnamed: 8': 'rx_acres'}, inplace=True)
    
    df.drop(index=0, inplace=True)
    
    df['state'] = df['state'].fillna(method='ffill')
    
    df = df.loc[df['state'].isin(west_states)]
    
    return df

### United into big ol function

In [44]:
# from scratch (all functions in here instead of defined elsewhere)

def pdf_no_more(input_path, output_path, year):

    pdf_list = os.listdir(input_path)
    
    west_states = ['CA', 'OR', 'WA', 'AZ', 'NV', 'UT', 'ID']
    
    df_list = []
    
    for i in pdf_list:
        filepath = input_path + i

        table = tabula.read_pdf(filepath, pages=1)
        df = table[0].copy()

        
        if int(filepath[-5]) == 1:
            print(f'processing page Alpha {filepath[-5]}')

            df.rename(columns = {'Unnamed: 0': 'state', 'Unnamed: 1':'agency', 'Wildland':'fires_acres', 'Unnamed: 2': 'rx_fires', 'Rx': 'rx_acres'}, inplace=True)
            df.drop(index=0, inplace=True)
            df[['wildfires', 'wild_acres']] = df['fires_acres'].str.split(expand=True)
            df.drop('fires_acres', axis=1, inplace=True)
            df['state'] = df['state'].fillna(method='ffill')
            df = df.loc[df['state'].isin(west_states)]
            
            df_list.append(df)

            
        if int(filepath[-5]) == 0: #page 10 will read as 0 - only last digit referenced
            
            print(f'processing page Omega {filepath[-5]}')
            df['Unnamed: 1'] = df['Unnamed: 1'].fillna('Totals')
            df.drop(columns=['Rx', 'Wildland', 'Unnamed: 10'], inplace=True)

            df['Unnamed: 2'].fillna(df['Unnamed: 3'], inplace=True)
            df['Unnamed: 4'].fillna(df['Unnamed: 5'], inplace=True)
            df['Unnamed: 6'].fillna(df['Unnamed: 7'], inplace=True)
            df['Unnamed: 8'].fillna(df['Unnamed: 9'], inplace=True)

            df.drop(columns=['Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7', 'Unnamed: 9'], inplace=True)

            df.rename(columns={'Unnamed: 0':'state', 'Unnamed: 1': 'agency', 'Unnamed: 2':'wildfires',
                             'Unnamed: 4':'wild_acres', 'Unnamed: 6':'rx_fires', 
                              'Unnamed: 8': 'rx_acres'}, inplace=True)

            df.drop(index=0, inplace=True)
            df['state'] = df['state'].fillna(method='ffill')
            df = df.loc[df['state'].isin(west_states)]
            
            df_list.append(df)
        
        if 1 < int(filepath[-5]) & int(filepath[-5]) < 10:

            print(f'processing page Medio {filepath[-5]}')

            df['Wildland'] = df['Wildland'].fillna('Totals')
            df.drop(columns=['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 10'], inplace=True)

            df['Rx'].fillna(df['Unnamed: 3'], inplace=True)
            df['Unnamed: 4'].fillna(df['Unnamed: 5'], inplace=True)
            df['Unnamed: 6'].fillna(df['Unnamed: 7'], inplace=True)
            df['Unnamed: 8'].fillna(df['Unnamed: 9'], inplace=True)

            df.drop(columns=['Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7', 'Unnamed: 9'], inplace=True)

            df.rename(columns={'Unnamed: 0':'state', 'Wildland': 'agency', 'Rx':'wildfires',
                             'Unnamed: 4':'wild_acres', 'Unnamed: 6':'rx_fires', 
                              'Unnamed: 8': 'rx_acres'}, inplace=True)

            df.drop(index=0, inplace=True)
            df['state'] = df['state'].fillna(method='ffill')
            df = df.loc[df['state'].isin(west_states)]
            
            df_list.append(df)
            
    final = pd.concat(df_list)
#     final.to_csv(f'{output_path}/annual_{year}_agency.csv', index=False)
    
#     final.loc[final['agency']=='Totals'].to_csv(f'{output_path}/annual_{year}_tot.csv', index=False)
    
    return final

In [45]:
input_path_1 = '../scratch_data/pdf_extracts/bystate_2010_extracted-pages/'
input_path_2 = '../scratch_data/pdf_extracts/bystate_2011_extracted-pages/'
input_path_3 = '../scratch_data/pdf_extracts/bystate_2012_extracted-pages/'
input_path_4 = '../scratch_data/pdf_extracts/bystate_2013_extracted-pages/'
output_path = '../scratch_data/pdf_extracts/resulting_csvs/'

In [51]:
#pdf_no_more(input_path_4, output_path, 2013) #breaks on 2013.  we had a good run.

## Construct 2013 manually
#### Another big format change coming 2014-2016, so not worth automating

In [61]:
AZ_2013 = p1_clean('../scratch_data/pdf_extracts/bystate_2013_extracted-pages/2013-1.pdf')
AZ_2013 #checked against doc

,state,agency,rx_fires,rx_acres,wildfires,wild_acres
22,AZ,BIA,13,"18,215",419,"47,132"
23,AZ,BLM,11,916,162,"8,164"
24,AZ,DDQ,0,0,1,10
25,AZ,FWS,1,23,28,"3,336"
26,AZ,NPS,2,236,62,"1,707"
27,AZ,OTHR,0,0,224,"8,721"
28,AZ,PRI,0,0,0,0
29,AZ,ST,0,0,128,"7,936"
30,AZ,USFS,107,"30,101",732,"28,275"
31,AZ,Totals,134,"49,491","1,756","105,281"


In [57]:
def midpages_2013(filename):

    west_states = ['CA', 'OR', 'WA', 'AZ', 'NV', 'UT', 'ID']
    table = tabula.read_pdf(filename, pages=1)
    
    df = table[0].copy()
    
    df.drop(columns=['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7'], inplace=True)
    
    df.rename(columns={'Unnamed: 0':'state', 'Wildland': 'agency', 'Rx':'wildfires',
                     'Unnamed: 4':'wild_acres', 'Unnamed: 6':'rx_fires', 
                      'Unnamed: 8': 'rx_acres'}, inplace=True)
    
    df.drop(index=0, inplace=True)
    
    df['state'] = df['state'].fillna(method='ffill')
    
    df = df.loc[df['state'].isin(west_states)]
    
    return df

In [106]:
path = '../scratch_data/pdf_extracts/bystate_2013_extracted-pages/'
pdf_list = os.listdir(path)

all_paths = [path + i for i in pdf_list] #order = 3, 2, 10, 1, 7, 8

p2 = midpages_2013(all_paths[1])
p3 = midpages_2013(all_paths[0])
#p7 = midpages_2013(all_paths[4])  #no states of interest on this page
p8 = midpages_2013('../scratch_data/pdf_extracts/bystate_2013_extracted-pages/2013-8.pdf')  #broken ^^

In [85]:
pdf = '../scratch_data/pdf_extracts/bystate_2013_extracted-pages/2013-10.pdf'
p10 = tabula.read_pdf(pdf, pages=1)
p10 = p10[0]
#p10

In [86]:
p10.drop(columns=['Wildland', 'Rx', 'Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7'], inplace=True)

p10.rename(columns={'Unnamed: 0':'state', 'Unnamed: 1': 'agency', 'Unnamed: 2':'wildfires',
                 'Unnamed: 4':'wild_acres', 'Unnamed: 6':'rx_fires', 
                  'Unnamed: 8': 'rx_acres'}, inplace=True)

p10.drop(index=0, inplace=True)

p10['state'] = p10['state'].fillna(method='ffill')

p10 = p10.loc[p10['state'].isin(west_states)]

p10

,state,agency,wildfires,wild_acres,rx_fires,rx_acres
16,UT,BIA,62,213,0,0
17,UT,BLM,417,"24,394",16,"1,048"
18,UT,DDQ,1,0,0,0
19,UT,FWS,0,0,0,0
20,UT,NPS,25,65,5,672
21,UT,PRI,1,"16,505",5,191
22,UT,ST,506,"5,011",35,"1,124"
23,UT,USFS,264,"24,094",50,"10,994"
24,UT,Totals,"1,276","70,282",111,"14,029"


In [68]:
#oh good, this one goes to 11
pdf = '../scratch_data/pdf_extracts/bystate_2013_extracted-pages/2013-11.pdf'
p11 = tabula.read_pdf(pdf, pages=1)
p11 = p11[0]  #I thiiink same as mediopages function?

In [70]:
p11_final = midpages_2013('../scratch_data/pdf_extracts/bystate_2013_extracted-pages/2013-11.pdf')
p11_final

,state,agency,wildfires,wild_acres,rx_fires,rx_acres
1,WA,BIA,349,"38,542",14,814
2,WA,BLM,38,"6,618",2,822
3,WA,CNTY,3,"4,510",0,0
4,WA,FWS,49,967,16,940
5,WA,NPS,42,264,12,241
6,WA,OTHR,0,0,0,0
7,WA,ST,741,"93,656",0,0
8,WA,USFS,305,"8,046",108,"10,328"
9,WA,Totals,"1,527","152,603",152,"13,145"


In [109]:
all_2013 = pd.concat([AZ_2013, p2, p3, p8, p10, p11_final])
all_2013.head()

,state,agency,rx_fires,rx_acres,wildfires,wild_acres
22,AZ,BIA,13,"18,215",419,"47,132"
23,AZ,BLM,11,916,162,"8,164"
24,AZ,DDQ,0,0,1,10
25,AZ,FWS,1,23,28,"3,336"
26,AZ,NPS,2,236,62,"1,707"


In [111]:
all_2013.to_csv(output_path + 'annual_2013_agency.csv', index=False)
all_2013.loc[all_2013['agency'] == 'Totals'].to_csv(output_path + 'annual_2013.tot.csv', index=False)

## 2014-2015 completely different formats than priors
Curses.  All pages unique.  Proceeding manually for all, last push.

In [112]:
filename14 = '../scratch_data/pdf_extracts/2014.pdf'
filename15 = '../scratch_data/pdf_extracts/2015.pdf'

In [118]:
def clean1_newformat(filename):
    west_states = ['CA', 'OR', 'WA', 'AZ', 'NV', 'UT', 'ID']
    table = tabula.read_pdf(filename, pages=1)
    df = table[0].copy()
    df.drop(columns=['Unnamed: 2', 'Unnamed: 4', 'Wildland', 
                     'Unnamed: 5', 'Unnamed: 8', 'Rx'], inplace=True)
    df.rename(columns = {'Unnamed: 0':'state', 'Unnamed: 1':'agency', 'Unnamed: 3':'wildfires',
                        'Unnamed: 6': 'wild_acres', 'Unnamed: 7': 'rx_fires', 
                         'Unnamed: 9': 'rx_acres'}, inplace=True)
    df['agency'] = df['agency'].fillna('Totals')
    df['state'] = df['state'].fillna(method = 'ffill')
    df = df.loc[df['state'].isin(west_states)]

    return df

In [120]:
p1_2014 = clean1_newformat(filename14)

In [121]:
def hopefully_more_than_one_page(file, page): #nope just one page.
    west_states = ['CA', 'OR', 'WA', 'AZ', 'NV', 'UT', 'ID']
    table = tabula.read_pdf(file, pages=page)
    df = table[0].copy()
    
    df['Unnamed: 2'] = np.where(df['Unnamed: 2']=='Totals', df['Unnamed: 11'], df['Unnamed: 2'])
    
    #manually fix rx_fires_total outside of function for P2 ONLY
    
    df.rename(columns = {'Unnamed: 0': 'state', 'Wildland':'agency', 'Rx':'wildfires',
                       'Unnamed: 1':'wild_acres', 'Unnamed: 2':'rx_fires',
                       'Unnamed: 3': 'rx_acres'}, inplace=True)
    
    df['wild_acres'].fillna(df['Unnamed: 8'], inplace=True)
    df['wildfires'].fillna(df['Unnamed: 5'], inplace=True)
    df['rx_acres'] = df.rx_acres.fillna(df['Unnamed: 14'])
    
    df['state'].fillna(method='ffill', inplace=True)
    df['agency'] = [i if i not in west_states else 'Totals' for i in df['agency']]
    
    df.drop(df.filter(regex='Unnamed').columns, axis=1, inplace=True)
    df.drop(index=0, inplace=True)
    df = df.loc[df['state'].isin(west_states)]
    
        
    return df

In [135]:
p2_2014 = hopefully_more_than_one_page(filename14, 2) 
p2_2014['rx_fires'][9] = 515 #note to manually fix units in rx_fires_tot

In [136]:
table = tabula.read_pdf(filename14, pages=3)
p3 = table[0]

In [139]:
#now that you've started cut-pasting intsead of generalizing a function,
#this chunk is good for 3 pages
p3['Unnamed: 2'] = np.where(p3['Unnamed: 2']=='Totals', p3['Unnamed: 11'], p3['Unnamed: 2'])
p3.rename(columns = {'Unnamed: 0': 'state', 'Unnamed: 1':'agency', 'Wildland':'wildfires',
                   'Rx':'wild_acres', 'Unnamed: 2':'rx_fires',
                   'Unnamed: 3': 'rx_acres'}, inplace=True)
p3['wild_acres'].fillna(p3['Unnamed: 8'], inplace=True)
p3['wildfires'].fillna(p3['Unnamed: 5'], inplace=True)
p3['rx_acres'] = p3.rx_acres.fillna(p3['Unnamed: 14'])
p3['state'].fillna(method='ffill', inplace=True)
p3['agency'] = [i if i not in west_states else 'Totals' for i in p3['agency']]
p3.drop(p3.filter(regex='Unnamed').columns, axis=1, inplace=True)
p3.drop(index=0, inplace=True)
p3 = p3.loc[p3['state'].isin(west_states)]

In [141]:
table = tabula.read_pdf(filename14, pages=7)
p7 = table[0]

In [144]:
p7['Unnamed: 2'] = np.where(p7['Unnamed: 2']=='Totals', p7['Unnamed: 11'], p7['Unnamed: 2'])
p7.rename(columns = {'Unnamed: 0': 'state', 'Unnamed: 1':'agency', 'Wildland':'wildfires',
                   'Rx':'wild_acres', 'Unnamed: 2':'rx_fires',
                   'Unnamed: 3': 'rx_acres'}, inplace=True)
p7['wild_acres'].fillna(p7['Unnamed: 8'], inplace=True)
p7['wildfires'].fillna(p7['Unnamed: 5'], inplace=True)
p7['rx_acres'] = p7.rx_acres.fillna(p7['Unnamed: 14'])
p7['state'].fillna(method='ffill', inplace=True)
p7['agency'] = [i if i not in west_states else 'Totals' for i in p7['agency']]
p7.drop(p7.filter(regex='Unnamed').columns, axis=1, inplace=True)
p7.drop(index=0, inplace=True)
p7 = p7.loc[p7['state'].isin(west_states)]

In [146]:
p7['rx_fires'][9] = 27
p7['rx_fires'][36] = 658

/Users/helenmeigs/miniforge3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [147]:
table = tabula.read_pdf(filename14, pages=9)
p9 = table[0]

In [149]:
p9['Unnamed: 2'] = np.where(p9['Unnamed: 2']=='Totals', p9['Unnamed: 11'], p9['Unnamed: 2'])
p9.rename(columns = {'Unnamed: 0': 'state', 'Unnamed: 1':'agency', 'Wildland':'wildfires',
                   'Rx':'wild_acres', 'Unnamed: 2':'rx_fires',
                   'Unnamed: 3': 'rx_acres'}, inplace=True)

p9['wild_acres'].fillna(p9['Unnamed: 8'], inplace=True)
p9['wildfires'].fillna(p9['Unnamed: 5'], inplace=True)
p9['rx_acres'] = p9.rx_acres.fillna(p9['Unnamed: 14'])

p9['state'].fillna(method='ffill', inplace=True)
p9['agency'] = [i if i not in west_states else 'Totals' for i in p9['agency']]

p9.drop(p9.filter(regex='Unnamed').columns, axis=1, inplace=True)
p9.drop(index=0, inplace=True)
p9 = p9.loc[p9['state'].isin(west_states)]

In [152]:
p9['rx_acres'][12] = 15801
p9['rx_acres'][33] = 15197

/Users/helenmeigs/miniforge3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [157]:
output_path

'../scratch_data/pdf_extracts/resulting_csvs/'

In [158]:
all_2014 = pd.concat([p1_2014, p2_2014, p3, p7, p9])
all_2014.to_csv(output_path + 'annual_2014_agency.csv', index=False)
all_2014.loc[all_2014.agency == 'Totals'].to_csv(output_path + 'annual_2014_tot.csv', index=False)

## 2015

In [182]:
pages = [1, 2, 3, 7, 9]

pdf_list=[]

for i in pages:
    table = tabula.read_pdf(filename15, pages=i) #filename15 defined above
    pdf_list.append(table[0])

In [184]:
p1 = pdf_list[0].copy()

p1['Unnamed: 1'].fillna('Totals', inplace=True)
p1.drop(['Unnamed: 2', 'Unnamed: 4', 'Wildland', 'Unnamed: 6', 'Unnamed: 8', 'Rx'],
       axis=1, inplace=True)
p1.rename(columns = {'Unnamed: 0':'state', 'Unnamed: 1': 'agency', 'Unnamed: 3': 'wildfires',
                    'Unnamed: 5': 'wild_acres', 'Unnamed: 7': 'rx_fires', 'Unnamed: 9': 'rx_acres'},
         inplace=True)
p1.state.fillna(method='ffill', inplace=True)
p1 = p1.loc[p1.state.isin(west_states)]
p1

,state,agency,wildfires,wild_acres,rx_fires,rx_acres
22,AZ,BIA,369,"56,361",23,"7,084"
23,AZ,BLM,147,"2,675",16,"3,976"
24,AZ,DOD,0,0,0,0
25,AZ,FWS,6,62,3,81
26,AZ,NPS,30,"3,745",7,485
27,AZ,OTHR,0,0,0,0
28,AZ,ST,404,"4,671",0,0
29,AZ,USFS,706,"92,638",175,"85,347"
30,AZ,Totals,"1,662","160,152",224,"96,973"


In [186]:
p2 = pdf_list[1].copy()

p2['Unnamed: 2'] = np.where(p2['Unnamed: 2']=='Totals', 617, p2['Unnamed: 2']) #hardcode rx_fires_tot
p2['Rx'].fillna(p2['Unnamed: 5'], inplace=True)
p2['Unnamed: 1'].fillna(p2['Unnamed: 8'], inplace=True)
p2['Unnamed: 3'] = p2['Unnamed: 3'].fillna(p2['Unnamed: 14'])
p2.rename(columns = {'Unnamed: 0': 'state', 'Wildland':'agency', 'Rx':'wildfires',
                   'Unnamed: 1':'wild_acres', 'Unnamed: 2':'rx_fires',
                   'Unnamed: 3': 'rx_acres'}, inplace=True)
p2['state'].fillna(method='ffill', inplace=True)
p2['agency'] = [i if i not in west_states else 'Totals' for i in p2['agency']]
p2.drop(p2.filter(regex='Unnamed').columns, axis=1, inplace=True)
p2.drop(index=0, inplace=True)
p2 = p2.loc[p2['state'].isin(west_states)]
p2

,state,agency,wildfires,wild_acres,rx_fires,rx_acres
1,CA,BIA,178,360,2,28
2,CA,BLM,97,"18,058",4,803
3,CA,C&L,4,15,1,30
4,CA,DOD,115,"11,394",0,0
5,CA,FWS,18,23,33,"8,462"
6,CA,NPS,126,"9,834",24,"1,208"
7,CA,OTHR,1,15,0,0
8,CA,ST,"6,550","316,217",0,0
9,CA,USFS,"1,656","537,446",553,"36,298"
10,CA,Totals,"8,745","893,362",617,"46,829"


In [188]:
p3 = pdf_list[2].copy()

p3['Unnamed: 2'] = np.where(p3['Unnamed: 2']=='Totals', p3['Unnamed: 11'], p3['Unnamed: 2'])
p3.rename(columns = {'Unnamed: 0': 'state', 'Unnamed: 1':'agency', 'Wildland':'wildfires',
                   'Rx':'wild_acres', 'Unnamed: 2':'rx_fires',
                   'Unnamed: 3': 'rx_acres'}, inplace=True)
p3['wild_acres'].fillna(p3['Unnamed: 8'], inplace=True)
p3['wildfires'].fillna(p3['Unnamed: 5'], inplace=True)
p3['rx_acres'] = p3.rx_acres.fillna(p3['Unnamed: 14'])
p3['state'].fillna(method='ffill', inplace=True)
p3['agency'] = [i if i not in west_states else 'Totals' for i in p3['agency']]
p3.drop(p3.filter(regex='Unnamed').columns, axis=1, inplace=True)
p3.drop(index=0, inplace=True)
p3 = p3.loc[p3['state'].isin(west_states)] #checked against doc

In [196]:
p7 = pdf_list[3].copy()

p7['Unnamed: 2'] = np.where(p7['Unnamed: 2']=='Totals', p7['Unnamed: 11'], p7['Unnamed: 2'])
p7.rename(columns = {'Unnamed: 0': 'state', 'Unnamed: 1':'agency', 'Wildland':'wildfires',
                   'Rx':'wild_acres', 'Unnamed: 2':'rx_fires',
                   'Unnamed: 3': 'rx_acres'}, inplace=True)
p7['wild_acres'].fillna(p7['Unnamed: 8'], inplace=True)
p7['wildfires'].fillna(p7['Unnamed: 5'], inplace=True)
p7['rx_acres'] = p7.rx_acres.fillna(p7['Unnamed: 14'])
p7['state'].fillna(method='ffill', inplace=True)
p7['agency'] = [i if i not in west_states else 'Totals' for i in p7['agency']]
p7.drop(p7.filter(regex='Unnamed').columns, axis=1, inplace=True)
p7.drop(index=0, inplace=True)
p7 = p7.loc[p7['state'].isin(west_states)]
p7['rx_fires'][9] = 36
p7['rx_fires'][36] = 551
p7 #checked against doc

,state,agency,wildfires,wild_acres,rx_fires,rx_acres
1,NV,BIA,3,2,0,0
2,NV,BLM,351,"17,747",10,700
3,NV,C&L,34,156,0,0
4,NV,FWS,7,0,2,722
5,NV,NPS,16,8,2,616
6,NV,OTHR,0,0,0,0
7,NV,ST,52,"5,048",18,"2,706"
8,NV,USFS,88,"19,518",4,172
9,NV,Totals,551,"42,479",36,"4,916"
28,OR,BIA,62,"71,804",2,90


In [199]:
p9 = pdf_list[4].copy()

p9['Unnamed: 2'] = np.where(p9['Unnamed: 2']=='Totals', p9['Unnamed: 11'], p9['Unnamed: 2'])
p9.rename(columns = {'Unnamed: 0': 'state', 'Unnamed: 1':'agency', 'Wildland':'wildfires',
                   'Rx':'wild_acres', 'Unnamed: 2':'rx_fires',
                   'Unnamed: 3': 'rx_acres'}, inplace=True)
p9['wild_acres'].fillna(p9['Unnamed: 8'], inplace=True)
p9['wildfires'].fillna(p9['Unnamed: 5'], inplace=True)
p9['rx_acres'] = p9.rx_acres.fillna(p9['Unnamed: 14'])
p9['state'].fillna(method='ffill', inplace=True)
p9['agency'] = [i if i not in west_states else 'Totals' for i in p9['agency']]
p9.drop(p9.filter(regex='Unnamed').columns, axis=1, inplace=True)
p9.drop(index=0, inplace=True)
p9 = p9.loc[p9['state'].isin(west_states)]
p9

,state,agency,wildfires,wild_acres,rx_fires,rx_acres
1,UT,BIA,30,23,0,0
2,UT,BLM,233,997,23,461
3,UT,DOD,0,0,0,0
4,UT,FWS,0,3,1,"1,060"
5,UT,NPS,14,55,7,118
6,UT,OTHR,0,0,11,"1,120"
7,UT,ST,456,"4,306",31,734
8,UT,USFS,197,"4,819",46,"15,538"
9,UT,Totals,930,"10,203",119.0,"19,031"
22,WA,BIA,290,"330,657",10,"4,436"


In [200]:
all_2015 = pd.concat([p1, p2, p3, p7, p9])

In [202]:
output_path

'../scratch_data/pdf_extracts/resulting_csvs/'

In [203]:
all_2015.reset_index(drop=True, inplace=True)
all_2015.to_csv(output_path+'annual_2015_agency.csv', index=False)
all_2015.loc[all_2015['agency']=='Totals'].to_csv(output_path+'annual_2015_tot.csv', index=False)

## Literally just type out 2016 like a plebe because it's a disaster
States split across pages